<a href="https://colab.research.google.com/github/Technology-for-the-Poorest-Billion/2025-Majicom-WaterCooling-passive/blob/main/GM2MajicomPassive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run to install libraries and initialise

In [3]:
!pip install CoolProp
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import CoolProp.CoolProp as CP
from CoolProp.HumidAirProp import HAPropsSI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 61.3 MB/s eta 0:00:00


Creates a set of classes to define various input parameters of interest: Air transport property, tank / kiosk geometry and fan behaviour

In [4]:
class AP: # Air Property
  def __init__(self,T,R, P = 101325):
    self.T = T # Temperature
    self.R = R # Relative Humidity
    self.P = P # Pressure

  def density(self):
    Vha = HAPropsSI('Vha', 'T', self.T, 'P', self.P, 'R', self.R)
    return 1/Vha

  def dynamicViscosity(self):
    return HAPropsSI('mu', 'T', self.T, 'P', self.P, 'R', self.R)

  def kinematicViscosity(self):
    return self.dynamic_viscosity()/self.density()

  def thermalConductivity(self):
    return HAPropsSI('k', 'T', self.T, 'P', self.P, 'R', self.R)

  def diffusionCoefficient(self):
    D0 = 2.5 * (10**-5)
    T0 = 1
    D = D0*(self.T/T0)**1.75

    return D


class fan: # fan properties
  def __init__(self,P, CFM):
    self.P = P # Power Consumption
    self.CFM = CFM # Mass flow rate (Cubic Feet per minute)


class tankGeo:
  def __init__(self,Router,Rinner,tankHeigh,t, k):
    self.Router = Router # Kiosk Shell Radius
    self.Rinner = Rinner # Tank Radius
    self.tankHeight = tankHeight
    self.t = t # Wall thickness
    self.k = k # Thermal Conductivity of wall material

  def areaFlow(self):
    return np.pi*(self.Router**2) - np.pi*(self.Rinner**2)

  def volumeTank(self):
    return np.pi*(self.Rinner**2)*self.tankHeight

  def areaFlow(self):
    return np.pi*(self.Router**2) - np.pi*(self.Rinner**2)

  def areaTank(self):
    return np.pi*(self.Rinner**2)*self.tankHeight

class mat: # material

  def __init__(self,k,t,msat,density,porosity):
    self.k = k # Thermal Conductivity when dry
    self.t = t # Thickness
    self.msat = msat # Mass of water at maximum saturation per unit area of material
    self.density = density # Density of material (dry)
    self.porosity = porosity # Porosity of material (fraction of the total volume that is empty pores)










In [5]:
def velocityAir(tank: tankGeo ,fan: fan):


  A = tank.areaFlow
  Q = fan.CFM*0.00047194745                           # Unit conversion between Cubic Feet per Minute and m^3/s
  v = Q/A
  return v

def hm(tank: tankGeo, fan:fan, air:AP):               # Defining the mass transfer coefficient for water into humid air

  L = tank.tankHeight

  v = velocityAir(tank,fan)
  Sc = air.kinematicViscosity/air.diffusionCoefficient # Schmidt Number
  Re = v*L/air.kinematicViscosity                      # Reynold's Number with tank height as characteristic length

  Sh = 0.664*(Re**0.5)*(Sc**(1/3))                     # Sherwood Number

  return Sh*air.diffusionCoefficient/L



def evaporationRate(tank: tankGeo, fan:fan, material: mat, air: AP, S ): # New Variable S is the mass saturation = mass of water / maximum mass of water

  massCoeff = hm(tank,fan,air)

  A = tank.areaTank
  densityWater = 1000 # kgm^3
  VTotal = tank.areaTank*material.t
  VWater = material.msat/densityWater # Total volume of water added
  VPore = material.porosity*VTotal # Volume of pores
  VMaterial = VTotal - VPore

  kWater = 0.6 #estimate for thermal conductivity of water
  hAir = 10    # convective heat loss coefficient for air (may need to change)
  hfg = 1 # Latent heat of evaporation




  ## Handling 3 Phase mixture conductivity

  if VPore > VWater: # Not all pores in the fibre are occupied so there is still some air remaining

    VAir = VPore - VWater
    kTotal = (material.k*VMaterial + kWater*VWater + air.thermalConductivity*VAir)/VTotal

  else: # The pores in the fibre are fully occupied with water and all air is displaced. Additional water can be absorbed by swelling

    kTotal = (material.k*VMaterial + kWater*VWater)/VTotal





  ## Defining the thermal resistances based on thermal conductivity


  Raf = 1/(hAir*A) #Thermal resistance from air to fabric
  Rfw = np.ln((tank.Rinner + mat.t) / tank.Rinner)/(2*np.pi()*tank.height*kTotal) + np.ln((tank.Rinner) / (tank.Rinner - tank.t) )/(2*np.pi()*tank.height*tank.k) #Thermal resistance from fabric to water. Currently to surface of water
  Rawc = tank.t/(tank.k*2*np.pi()*(tank.Rinner**2))
  Rawe = 0 # emissive resistance (0 is just a filler for now)
  Raw = Rawc + Rawe # Likely temporary probably better to handle any radiative input as a current source into the water

  ## Iterating to solve for Tf, Tw and Pv

  Tf = air.T - 2 # Starting guess, slightly below ambient

  tol = 1e-3
  maxRunTime = 100

  for i in range(maxRunTime):

    Tw = (Rfw * air.T + Raw * Tf)/(Raw + Rfw)
    Pv = 1 #evaluate Pv at Tf
    Ps = 1 #evaluate Ps at Ta
    mdot = S*massCoeff*A*(Pv - Ps)

    q_evap = mdot*hfg

    qaf = (air.T - Tf)/Raf
    qwf = (Tw - Tf)/Rfw

    e = qaf + qwf - q_evap

    if abs(e) < tol:
      break

    Tf -= 0.1*e #Realaxtion method of shifting by fixed proportion of error. Can upgrade to Newton-Raphson


  return Tw









